# Tennis Grand Slam

Load packages

In [1]:
import pandas as pd

#ignore warnings
import warnings
warnings.filterwarnings('ignore') 

#pandas defaults
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)

Get data from wikipedia

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_Grand_Slam_men%27s_singles_champions')[2]

Prepare data

In [3]:
# clean table
melt = pd.melt(df, id_vars=['Year'], value_name='metric')
melt[['Player', 'titles']] = melt['metric'].str.split('(', expand=True) 

# keep only Big 3
data = melt[melt['Player'].isin(['Rafael Nadal ', 'Roger Federer ', 'Novak Djokovic '])]

# number of wins running total
data['win'] = 1
data['sum'] = data.groupby(['Player', 'variable'])['win'].cumsum()

# keep relevant field
data = data[['Year', 'variable', 'win', 'Player', 'sum']]

# add Pete Sampras data
d = {'Year':2002, 'variable':['Australian Open', 'Wimbledon', 'US Open'], 'sum':[2,7,5], 'Player': 'Pete Sampras'}
ps = pd.DataFrame(d)
data = pd.concat([ps,data])

In [4]:
# create combination of all possibilities between palyers, tournments and years

# player
p = pd.DataFrame(data['Player'].unique(), columns=['Player'])
p['join'] = 'j'

#tournment
t = pd.DataFrame(data['variable'].unique(),columns=['variable'])
t['join'] = 'j'

# year
y = pd.DataFrame(data['Year'].unique(),columns=['Year'])
y['join'] = 'j'

# join everything
join = pd.merge(p, t, on='join', how='inner')
join = pd.merge(join, y, on='join', how='inner')

In [5]:
# finalize preparation
fill = pd.merge(data, join, on=['Player', 'variable', 'Year'], how='right').sort_values(['Player','variable', 'Year'])
fill["all_time"] = fill.groupby(['Player', 'variable'])['sum'].transform(lambda x: x.ffill())
fill[['win', 'all_time']] = fill[['win', 'all_time']].fillna(0).astype(int)

# select relevant fields
final = fill[['Player', 'variable', 'Year', 'win', 'all_time']]

Save data

In [6]:
final.to_csv('grand slam.csv')
final.head()

,Player,variable,Year,win,all_time
189,Novak Djokovic,Australian Open,2002,0,0
199,Novak Djokovic,Australian Open,2003,0,0
190,Novak Djokovic,Australian Open,2004,0,0
197,Novak Djokovic,Australian Open,2005,0,0
191,Novak Djokovic,Australian Open,2006,0,0
